In [2]:
#importing necessary modules
from datetime import datetime as dt
import pandas as pd
import numpy as np
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt

import metpy.calc as mpcalc
import metpy.plots as mpplots

from matplotlib.patheffects import withStroke
from metpy.io import parse_metar_file
from metpy.units import pandas_dataframe_to_unit_arrays
from metpy.cbook import get_test_data
from metpy.units import units

from scipy.stats import zscore
 
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Link to datafile: https://drive.google.com/file/d/1kjzuaNx7G6EmjFbIToN6DitoZnbmfy9V/view?usp=sharing
# Open the netCDF file as a xarray Dataset
dataX = xr.open_dataset('C:\\Users\\Dell\\Documents\\BD final proj\\projdata1.nc4', decode_times = False)


# View a summary of the Dataset
dataX

TypeError: dropna() missing 1 required positional argument: 'dim'

In [ ]:
print(dataX.head,'\n\n\n\nOriginal Info:\n\n',dataX.info)

In [ ]:
#Converting Data to Pandas
data1 = dataX.to_dataframe().reset_index()

In [ ]:
#Renaming the column Headers
data = data1.rename(columns = {'lat':'Latitude', 'lon':'Longitude', 'irrx':'Irradiation','pptx':'Precipitation (mm/month)','tmin':'Min. Temp (C)','tmax':'Max Temp (C)','rhum':'Relative Humidity', 'petx': 'Potential Evapotranspiration','vpxx':'Vapor Pressure (Pa)', 'srad':'Solar Radiation', 'pptx':'Precipitation'})

In [ ]:
#Removing the unneeded columns from data
data.drop(['Potential Evapotranspiration'], axis = 1, inplace = True)

In [ ]:
#Dropping NaN values
data = data.dropna()


In [ ]:
#Removing outliers from each columnn - if values are more than 3 std deviations from the mean, they will be removed
z = zscore(data)
abs_z = np.abs(z)
filtered_data = (abs_z < 3).all(axis=1)
dataA = data[filtered_data]

In [ ]:
dataA

In [ ]:
print('\nNumber of rows & columns, respectively:\n\n\tUnfiltered Data:',data1.shape, '\n\tFiltered Data:',dataA.shape,'\n\n\nNumber of Elements:\n\n\tUnfiltered Data:',data1.size,'\n\tFilteredData:',dataA.size,'\n')

In [ ]:
dataXR = dataX.metpy.parse_cf()

In [ ]:
#temp = tmax.metpy.time
#hum = rhum.metpy.time
hum = dataXR.rhum.metpy.sel(time, method = 'nearest').squeeze()
temp = dataXR.tmin.metpy.sel(time, method='nearest').squeeze()